In [2]:
import os
import json
import ast
import numpy as np
import pandas as pd
from collections import Counter
pd.set_option('display.max_colwidth', 900)

In [3]:
def get_cardset(drafts):
    cardset = set()
    for draft in drafts:
        for pack in draft['picks']:
            for card in pack:
                cardset.add(card)
    carddict = {card: i for i, card in enumerate(cardset)}
    carddict['<unk>'] = -1
    return carddict

In [4]:
def parse_draft_to_flooey_format(draft):
    players = draft['picks']
    cards_per_pack = int(len(players[0]) / 3)
    packs = []
    for pack_index in range(3):
        choices = []
        for pick_index in range(cards_per_pack):
            options = []
            idx = pick_index
            while idx < cards_per_pack:
                options.append(players[(idx - pick_index) % 8][idx + pack_index * cards_per_pack])
                idx += 1
            choices.append(options)
        packs.append(choices)
    return packs

In [5]:
def vectorize_drafts(drafts):
    cardset = get_cardset(drafts)
    flooey_drafts = [parse_draft_to_flooey_format(draft) for draft in drafts]
    return [vectorize_draft(draft, cardset) for draft in flooey_drafts]

def vectorize_draft(draft, cardset):
    draft_x = np.zeros([3, len(draft[0][0]), len(cardset) * 2])
    draft_y = np.zeros([3, len(draft[0][0]), len(cardset)])
    pile = np.zeros(len(cardset))
    for pa, pack in enumerate(draft):
        for pi, pick in enumerate(pack):
            draft_y[pa][pi][cardset.get(pick[0], -1)] = 1
            x1 = np.zeros(len(cardset))
            for card in pick:
                if card in cardset:
                    x1[cardset[card]] += 1
                else:
                    # print("Unknown card:", card)
                    x1[-1] += 1
            draft_x[pa][pi] = np.append(x1, pile)
            pile[cardset.get(pick[0], -1)] += 1
            
    return {'draft_x': draft_x, 'draft_y': draft_y}

In [6]:
def draft_to_df(draftx, drafty):
    picks = []
    piles = []
    selections = []
    for (dx, dy) in zip(draftx, drafty):
        pick = []
        pile = []
        for card in np.where(dx != 0)[0]:
            if card < len(cardset):
                for i in range(int(dx[card])):
                    pick.append(invcardset[card])
            else:
                for i in range(int(dx[card])):
                    pile.append(invcardset[card - len(cardset)])
        picks.append(pick)
        piles.append(pile)
        selections.append(invcardset[np.where(dy==1)[0][0]])
    return pd.DataFrame({'picks': picks, 'piles': piles, 'selections': selections})

In [9]:
def load_draft(draft_file):
    return ast.literal_eval(open('drafts/' + mtgset + '/' + draft_file, 'r').read())

In [10]:
mtgset = 'DOM'
alldrafts = os.listdir('drafts/' + mtgset)
drafts = [load_draft(draft) for draft in alldrafts[:100]]
cardset = get_cardset(drafts)
invcardset = {v: k for k, v in cardset.items()}

In [13]:
drafts[0]

{'format': 'DOM',
 'picks': [['The_Eldest_Reborn',
   'Sylvan_Awakening',
   'Baloth_Gorger',
   'Urgoros_the_Empty_One',
   'Cabal_Paladin',
   'Windgrace_Acolyte',
   'Dark_Bargain',
   'Blessing_of_Belzenlok',
   'Stronghold_Confessor',
   'Arcane_Flight',
   'Stronghold_Confessor',
   'Guardians_of_Koilos',
   'Rat_Colony',
   'Tragic_Poet',
   'Settle_the_Score',
   'Windgrace_Acolyte',
   'Wild_Onslaught',
   "Navigator's_Compass",
   'Deathbloom_Thallid',
   'Grow_from_the_Ashes',
   'Grow_from_the_Ashes',
   'Slimefoot_the_Stowaway',
   'Saproling_Migration',
   'Broken_Bond',
   'Jousting_Lance',
   'Garna_the_Bloodflame',
   'Feral_Abomination',
   'Rat_Colony',
   'Vicious_Offering',
   'Cast_Down',
   'Dark_Bargain',
   'Fungal_Infection',
   'Spore_Swarm',
   'Slimefoot_the_Stowaway',
   'Bloodtallow_Candle',
   'Soul_Salvage',
   'Skittering_Surveyor',
   "Gaea's_Protector",
   'Benalish_Honor_Guard',
   'Demonic_Vigor',
   "Warlord's_Fury",
   'Cabal_Stronghold'],
  ['Fi

In [12]:
parse_draft_to_flooey_format(drafts[0])

[[['The_Eldest_Reborn',
   'Settle_the_Score',
   'Baird_Steward_of_Argive',
   "Navigator's_Compass",
   'Thallid_Omnivore',
   'Sparring_Construct',
   'Llanowar_Envoy',
   'Hinterland_Harbor',
   'Stronghold_Confessor',
   'Serra_Disciple',
   'Cabal_Evangel',
   'Arbor_Armament',
   'Cloudreader_Sphinx',
   'Ghitu_Lavarunner'],
  ['Sylvan_Awakening',
   'Run_Amok',
   'Call_the_Cavalry',
   'Llanowar_Envoy',
   'Demonic_Vigor',
   'Fervent_Strike',
   'Broken_Bond',
   'Weight_of_Memory',
   'Arcane_Flight',
   'Slinn_Voda_the_Rising_Deep',
   "Artificer's_Assistant",
   'Arbor_Armament',
   'Unwind'],
  ['Baloth_Gorger',
   "On_Serra's_Wings",
   'Aven_Sentry',
   'Keldon_Overseer',
   'Memorial_to_Folly',
   'Guardians_of_Koilos',
   'Primordial_Wurm',
   'Hinterland_Harbor',
   'Stronghold_Confessor',
   'Relic_Runner',
   'Cold-Water_Snapper',
   'Tolarian_Scholar'],
  ['Urgoros_the_Empty_One',
   'Shivan_Fire',
   "Gideon's_Reproach",
   'Ancient_Animus',
   'Naban_Dean_of_Ite

In [8]:
test = alldrafts[:1000]
train = alldrafts[1000:]

In [9]:
d = vectorize_draft(
    parse_draft_to_flooey_format(load_draft(train[0])),
    cardset
)

In [10]:
d['draft_x'].shape, d['draft_y'].shape

((3, 14, 500), (3, 14, 250))

In [45]:
def DraftGenerator(trainset, batch_size=20):
    for i in range(0, len(trainset) / batch_size):
        flooey_drafts = [
            parse_draft_to_flooey_format(load_draft(draft))
            for draft in trainset[i*batch_size:(i+1)*batch_size]
        ]
        draftsv = [vectorize_draft(draft, cardset) for draft in flooey_drafts]
        X, Y = [], []
        for draft in draftsv:
            draft_x = draft['draft_x']
            draft_y = draft['draft_y']
            for pack in range(len(draft_x)):
                for pick in range(len(draft_x[pack])):
                    Y.append(draft_y[pack][pick])
                    X.append(draft_x[pack][pick].flatten())
        yield (np.array(X), np.array(Y))

In [46]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [47]:
model = Sequential()
model.add(Dense(
    512, activation='relu',
    # input_shape=[3, 14, len(cardset) * 2])
    input_dim=len(cardset) * 2
))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(len(cardset), activation='softmax'))
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [51]:
dg = DraftGenerator(train)

In [52]:
model.fit_generator(dg, steps_per_epoch=2000, epochs=5)

Epoch 1/5
 120/2000 [>.............................] - ETA: 2:37 - loss: 1.1612 - acc: 0.5704

KeyboardInterrupt: 